In [0]:
!unzip /content/train.csv.zip
!unzip /content/test.csv.zip

Archive:  /content/train.csv.zip
  inflating: train.csv               
Archive:  /content/test.csv.zip
  inflating: test.csv                


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [0]:
df = pd.read_csv('/content/train.csv')

In [3]:
df.head(10)

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.000000,4,47
5,59859,0.666667,ur a sh*tty comment.,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:50.865549+00,2,NaN,2006,rejected,0,0,0,0,0,0.009524,0,105
6,59861,0.457627,hahahahahahahahhha suck it.,0.050847,0.305085,0.000000,0.254237,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:52.451277+00,2,NaN,2006,rejected,0,0,0,0,0,0.220339,0,59
7,59863,0.000000,FFFFUUUUUUUUUUUUUUU,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:54.055221+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
8,239575,0.000000,The ranchers seem motivated by mostly by greed...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-13 18:01:05.156229+00,6,NaN,26662,approved,0,0,0,0,0,0.000000,0,4
9,239576,0.000000,It was a great show. Not a combo I'd of expect...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-13 18:11:21.570460+00,6,239522.0,26650,approved,0,0,0,1,0,0.000000,0,4


In [0]:
y = df['target']
x = df['comment_text']

In [0]:
x = x.to_numpy()

In [6]:
x

array(["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
       "Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",
       'This is such an urgent design problem; kudos to you for taking it on. Very impressive!',
       ..., 'thank you ,,,right or wrong,,, i am following your advice',
       'Anyone who is quoted as having the following exchange, even if apocryphal, would have received my vote! \n\nBessie Braddock: "Winston, you are drunk, and what’s more you are disgustingly drunk."\nWinston Churchill: "Bessie, my dear, you are ugly, and what’s more, you are disgustingly ugly. But tomorrow I shall be sober and you will still be disgustingly ugly."',
       'Students defined as EBD are legally just as disabled and eligible for special services as a developmentally disabled or physically disabled student. \n\nEMOTIONAL AND BEHAVIORAL DISORDER (EBD).\nDefinition. \nAn 

In [7]:
from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [0]:
toknizer=text.Tokenizer(num_words=100000, lower=True)

In [0]:
toknizer.fit_on_texts(x)

In [0]:
word_index = toknizer.word_index

In [0]:
x_list = x.tolist()

In [0]:
train = toknizer.texts_to_sequences(x_list)

In [13]:
train

[[18,
  6,
  33,
  2259,
  62,
  50,
  48691,
  10,
  103,
  35,
  1156,
  2,
  192,
  18,
  1023,
  120,
  165,
  361,
  95,
  218],
 [408,
  10,
  18,
  40,
  93,
  60,
  170,
  5,
  223,
  221,
  6628,
  17555,
  185,
  12,
  54,
  3,
  57,
  243,
  204,
  61,
  7,
  35,
  96],
 [18, 6, 150, 44, 9626, 2892, 171, 4545, 2, 10, 9, 432, 12, 16, 101, 5008],
 [6, 18, 164, 633, 15, 402, 2, 5854, 16, 60, 1068, 53, 29, 10, 15, 6575, 12],
 [5388, 10, 726, 13, 5, 1098, 4, 2723],
 [7174, 5, 8029, 68643, 260],
 [3632, 12],
 [],
 [1,
  12729,
  393,
  3979,
  32,
  1115,
  32,
  2468,
  38,
  48,
  63,
  17,
  1,
  84,
  2,
  654,
  31,
  1614,
  1516,
  143,
  485],
 [12, 22, 5, 165, 341, 14, 5, 17661, 570, 4, 1553, 2, 15, 81, 784, 24, 12, 22],
 [791, 8, 657, 165],
 [18,
  6,
  5,
  165,
  292,
  224,
  11,
  437,
  26,
  1,
  222,
  37,
  14523,
  1184,
  1,
  10998,
  54,
  36,
  106,
  208,
  650,
  12],
 [18, 262, 50, 5, 935, 7, 1, 84, 1890],
 [62,
  1091,
  8,
  86,
  726,
  13,
  92,
  313

In [0]:
train = pad_sequences(train, maxlen=220)

In [15]:
train

array([[    0,     0,     0, ...,   361,    95,   218],
       [    0,     0,     0, ...,     7,    35,    96],
       [    0,     0,     0, ...,    16,   101,  5008],
       ...,
       [    0,     0,     0, ...,  1105,    35,  1817],
       [    0,     0,     0, ...,    15, 24264,  2584],
       [    0,     0,     0, ...,     5,  1676,   635]], dtype=int32)

In [16]:
train.shape

(1804874, 220)

In [17]:
y.head()

0    0.000000
1    0.000000
2    0.000000
3    0.000000
4    0.893617
Name: target, dtype: float64

In [0]:
def y_to_cat(x):
  if x >= 0.5:
    return 1
  else :
    return 0

In [0]:
y_label = y.apply(y_to_cat)

In [0]:
y_label = y_label.values

In [0]:
y_label = y_label.reshape(-1,1)

In [0]:
from keras import models, layers
from sklearn.model_selection import train_test_split
from keras import callbacks
from tensorboardcolab import  *

In [23]:
tbc = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://bc02d7a1.ngrok.io


In [0]:
X_train, X_test, y_train, y_test = train_test_split(train, y_label, test_size=0.2, random_state=1228)

In [25]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1443899, 220) (1443899, 1)
(360975, 220) (360975, 1)


In [26]:
input_tensor = layers.Input(shape=(220,))
embedding = layers.Embedding(100000+1, 300, input_length=220)(input_tensor)

hidden_1_1 = layers.Bidirectional(layers.CuDNNLSTM(64, return_sequences=True))(embedding)
hidden_1_2 = layers.Bidirectional(layers.CuDNNLSTM(128))(hidden_1_1)

hidden_2_1 = layers.Conv1D(64, 7, activation='relu')(embedding)
hidden_2_2 = layers.Conv1D(128, 7, activation='relu')(hidden_2_1)
hidden_2_3 = layers.Conv1D(256, 7, activation='relu')(hidden_2_2)
hidden_2_4 = layers.GlobalAveragePooling1D()(hidden_2_3)

concat = layers.Concatenate()([hidden_1_2, hidden_2_4])

hidden_dense_1 = layers.Dense(256, activation='relu')(concat)
hidden_dense_2 = layers.Dense(128, activation='relu')(hidden_dense_1)
output = layers.Dense(1, activation='sigmoid')(hidden_dense_2)




W0817 07:29:32.428830 139762859091840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0817 07:29:32.447202 139762859091840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0817 07:29:32.455976 139762859091840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [0]:
embedding_model = models.Model(input_tensor, output)

In [28]:
embedding_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 220)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 220, 300)     30000300    input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 214, 64)      134464      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 208, 128)     57472       conv1d_1[0][0]                   
__________________________________________________________________________________________________
bidirectio

In [0]:
# from keras.losses import 

In [30]:
embedding_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

W0817 07:29:33.809817 139762859091840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0817 07:29:33.852647 139762859091840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0817 07:29:33.860699 139762859091840 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [31]:
embedding_model.fit(X_train, y_train, batch_size=1024, epochs=50, validation_data=(X_test, y_test), callbacks=[TensorBoardColabCallback(tbc)])

W0817 07:29:34.942556 139762859091840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0817 07:29:35.776756 139762859091840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tensorboardcolab/core.py:49: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Train on 1443899 samples, validate on 360975 samples


W0817 07:29:37.455679 139762859091840 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/callbacks.py:850: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.



Epoch 1/50
1443899/1443899 [==============================] - 1527s 1ms/step - loss: 0.1446 - acc: 0.9462 - val_loss: 0.1335 - val_acc: 0.9472
Epoch 2/50
1443899/1443899 [==============================] - 1524s 1ms/step - loss: 0.1167 - acc: 0.9532 - val_loss: 0.1258 - val_acc: 0.9507
Epoch 3/50
1443899/1443899 [==============================] - 1522s 1ms/step - loss: 0.1023 - acc: 0.9583 - val_loss: 0.1312 - val_acc: 0.9490
Epoch 4/50
1443899/1443899 [==============================] - 1521s 1ms/step - loss: 0.0845 - acc: 0.9656 - val_loss: 0.1481 - val_acc: 0.9464
Epoch 5/50
 177152/1443899 [==>...........................] - ETA: 20:37 - loss: 0.0575 - acc: 0.9772

KeyboardInterrupt: ignored

In [0]:
from keras.callbacks import EarlyStopping

In [0]:
stop = EarlyStopping(monitor = 'val_acc', patience=5)

In [0]:
test = pd.read_csv('/content/test.csv')

In [33]:
test.head()

,id,comment_text
0,7097320,[ Integrity means that you pay your debts.]\n\...
1,7097321,This is malfeasance by the Administrator and t...
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...
3,7097323,"Paul: Thank you for your kind words. I do, in..."
4,7097324,Sorry you missed high school. Eisenhower sent ...


In [0]:
test_text = test['comment_text']

In [35]:
test_text

0        [ Integrity means that you pay your debts.]\n\...
1        This is malfeasance by the Administrator and t...
2        @Rmiller101 - Spoken like a true elitist. But ...
3        Paul: Thank you for your kind words.  I do, in...
4        Sorry you missed high school. Eisenhower sent ...
5        Let’s see if I understand this; Berkowitz anno...
6        Our oils read;  President IS taking different ...
7        'Work together'? Dream on. The liberals are al...
8        What would Jerry Prevo think about that story?...
9        When wil the  indigenous  be accountable and t...
10       the people against O'Leary are the same people...
11       Between racist Eastman and 'Slappy' Wilson the...
12       Since whining is the lifeblood of online comme...
13       Sue blue jeans, get out more and learn about t...
14       This debate will never end.  However the real ...
15       IT IS DOA! As Collins announced her intentions...
16       There are no words for these senseless acts.. .

In [0]:
test_text_list = test_text.to_list()

In [37]:
test_text_list

['[ Integrity means that you pay your debts.]\n\nDoes this apply to President Trump too?',
 'This is malfeasance by the Administrator and the Board.  They are wasting our money!',
 '@Rmiller101 - Spoken like a true elitist. But look out bud. The re-awakening in Europe, Brexit and now the Trump victory are just the beginning. The unwashed masses are coming for you and your ilk….',
 'Paul: Thank you for your kind words.  I do, indeed, have strong beliefs and don\'t hide them.  They are based on both facts and values.  I think the question of "bias" and "prejudice" comes into play when we\'re discussing beliefs and attitudes based on values.  One of our poster friends,  for example, often comments that being "intolerant" of a bias, for example, is, in itself, a bias.  This is, IMO, where the concept of "irratonal antipathy" is important.  If I dislike neo-Nazis because of their views on race, my antipathy is based on fact (knowledge of their views) and values (my belief that condemning ot

In [0]:
test_token = toknizer.texts_to_sequences(test_text_list)

In [0]:
test_pad = pad_sequences(test_token, maxlen = 220)

In [41]:
test_pad

array([[    0,     0,     0, ...,   152,    55,   116],
       [    0,     0,     0, ...,  3141,    56,   107],
       [    0,     0,     0, ...,    10,     3,    35],
       ...,
       [    0,     0,     0, ..., 24294,     7,   889],
       [    0,     0,     0, ...,     5,    81,   160],
       [    0,     0,     0, ...,   112,    11,   261]], dtype=int32)

In [0]:
y_pred = embedding_model.predict(test_pad)

In [43]:
y_pred

array([[9.2974305e-04],
       [3.8125753e-02],
       [1.2339252e-01],
       ...,
       [9.5538950e-01],
       [4.5003030e-01],
       [3.1751394e-04]], dtype=float32)

In [44]:
!unzip /content/sample_submission.csv.zip

Archive:  /content/sample_submission.csv.zip
  inflating: sample_submission.csv   


In [0]:
submit = pd.read_csv('/content/sample_submission.csv')

In [47]:
submit.head()

,id,prediction
0,7097320,0.0
1,7097321,0.0
2,7097322,0.0
3,7097323,0.0
4,7097324,0.0


In [0]:
submit['prediction'] = y_pred

In [49]:
submit.head()

,id,prediction
0,7097320,0.000930
1,7097321,0.038126
2,7097322,0.123393
3,7097323,0.000142
4,7097324,0.000073
